<a href="https://www.kaggle.com/code/mahmoudwalied2000/task-nlp?scriptVersionId=228742541" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install gensim tensorflow nltk tqdm scikit-learn pandas


In [2]:
import os
print(os.listdir("/kaggle/input/data-33"))  # عرض جميع الملفات داخل Kaggle Datasets


['IMDB Dataset.csv']


In [3]:
import numpy as np
import pandas as pd
import re
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
import gensim.downloader as api

# تحميل قائمة التوقف من NLTK
nltk.download('stopwords')

# تهيئة الأدوات
ps = PorterStemmer()
stop_words = set(stopwords.words('english'))

# تحديد الكلمات المهمة التي لا يجب إزالتها
important_words = {"great", "excellent", "amazing", "love", "good", "best", "awesome", "fantastic", "superb"}

# الاحتفاظ بالكلمات السلبية أثناء المعالجة
negative_words = {"bad", "boring", "weak", "terrible", "awful", "disappointing", 
                  "poor", "waste", "dull", "predictable", "horrible", "mediocre"}

# دالة تنظيف النصوص
def preprocess_review(review):
    review = re.sub(r'[^a-zA-Z\s]', '', review)  # إزالة الرموز الخاصة
    review = review.lower().split()
    review = [word for word in review if (word not in stop_words or word in important_words or word in negative_words)]
    return ' '.join(review)


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
import os
print(os.listdir("/kaggle/input/data-33"))


['IMDB Dataset.csv']


In [5]:
import pandas as pd
import re

# تحديد المسار الصحيح للملف داخل Kaggle
file_path = "/kaggle/input/data-33/IMDB Dataset.csv"  # تأكد من صحة المسار

# قراءة البيانات
df = pd.read_csv(file_path)

# تحويل المشاعر إلى أرقام: إيجابي = 1 ، سلبي = 0
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})

# تعريف قائمة الكلمات الممنوعة (stop words)
stop_words = set(["the", "is", "in", "and", "to", "of", "this", "it", "that", "on", "for", "with", "as", "was"])  # أضف المزيد حسب الحاجة

# تعريف الكلمات المهمة التي لا يجب إزالتها
important_words = set(["great", "excellent", "best", "amazing", "love", "good", "bad", "worst", "awful", "terrible"])  # أضف الكلمات التي تريد الاحتفاظ بها

# دالة تنظيف النصوص
def preprocess_review(review):
    review = re.sub(r'[^a-zA-Z\s]', '', review)  # إزالة الرموز الخاصة
    review = review.lower().split()
    review = [word for word in review if (word not in stop_words or word in important_words)]
    return ' '.join(review)

# التحقق من وجود عمود 'review' قبل تطبيق المعالجة النصية
if 'review' in df.columns:
    df['processed_review'] = df['review'].apply(preprocess_review)
else:
    print("⚠️ خطأ: لم يتم العثور على عمود 'review' في البيانات!")

# عرض أول 5 صفوف للتأكد من التحميل الصحيح
print(df.head())

# عرض عدد التقييمات لكل فئة مشاعر (0 = سلبي, 1 = إيجابي)
print(df['sentiment'].value_counts())


                                              review  sentiment  \
0  One of the other reviewers has mentioned that ...          1   
1  A wonderful little production. <br /><br />The...          1   
2  I thought this was a wonderful way to spend ti...          1   
3  Basically there's a family where a little boy ...          0   
4  Petter Mattei's "Love in the Time of Money" is...          1   

                                    processed_review  
0  one other reviewers has mentioned after watchi...  
1  a wonderful little production br br filming te...  
2  i thought a wonderful way spend time a too hot...  
3  basically theres a family where a little boy j...  
4  petter matteis love time money a visually stun...  
sentiment
1    25000
0    25000
Name: count, dtype: int64


In [6]:
# تحميل GloVe 100D
glove_model = api.load("glove-wiki-gigaword-100")

# تحويل النصوص إلى متجهات باستخدام GloVe
def get_sentence_vector_glove(sentence, model):
    word_vectors = [model[word] for word in sentence.split() if word in model]
    return np.mean(word_vectors, axis=0) if word_vectors else np.zeros(100)

X_glove = np.array([get_sentence_vector_glove(sent, glove_model) for sent in df['processed_review']])
y = np.array(df['sentiment'])  # 1 = إيجابي، 0 = سلبي


[==================================================] 100.0% 128.1/128.1MB downloaded


In [7]:
# تقسيم البيانات إلى تدريب واختبار
X_glove_train, X_glove_test, y_train, y_test = train_test_split(X_glove, y, test_size=0.2, random_state=42)

# إعادة تشكيل البيانات لـ LSTM
X_glove_train = X_glove_train.reshape((X_glove_train.shape[0], 1, X_glove_train.shape[1]))
X_glove_test = X_glove_test.reshape((X_glove_test.shape[0], 1, X_glove_test.shape[1]))


In [8]:
# بناء النموذج باستخدام Bidirectional LSTM
model = Sequential([
    Bidirectional(LSTM(128, return_sequences=True), input_shape=(1, 100)),
    Dropout(0.3),
    Bidirectional(LSTM(64, return_sequences=False)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

# تجميع النموذج
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# تدريب النموذج
model.fit(X_glove_train, y_train, epochs=15, batch_size=32, validation_split=0.2)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 14s 8ms/step - accuracy: 0.7003 - loss: 0.5627 - val_accuracy: 0.7789 - val_loss: 0.4672
Epoch 2/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7838 - loss: 0.4628 - val_accuracy: 0.7966 - val_loss: 0.4479
Epoch 3/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7861 - loss: 0.4619 - val_accuracy: 0.7883 - val_loss: 0.4559
Epoch 4/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.7871 - loss: 0.4573 - val_accuracy: 0.7946 - val_loss: 0.4505
Epoch 5/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7884 - loss: 0.4532 - val_accuracy: 0.8016 - val_loss: 0.4337
Epoch 6/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7945 - loss: 0.4460 - val_accuracy: 0.8021 - val_loss: 0.4324
Epoch 7/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.7916 - loss: 0.4454 - val_accuracy: 0.8034 - val_loss: 0.4285
Epoch 8/15
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - accuracy: 0.8000 - loss: 0.4354 -

In [9]:
from sklearn.metrics import accuracy_score, classification_report

# توقع المشاعر على بيانات الاختبار
y_pred = model.predict(X_glove_test)
y_pred = (y_pred > 0.5).astype(int)

# طباعة دقة النموذج والتقرير التفصيلي
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
Accuracy: 0.8033
              precision    recall  f1-score   support

           0       0.79      0.82      0.80      4961
           1       0.81      0.79      0.80      5039

    accuracy                           0.80     10000
   macro avg       0.80      0.80      0.80     10000
weighted avg       0.80      0.80      0.80     10000



In [10]:
def predict_sentiment(review, model, vectorizer):
    processed_review = preprocess_review(review)
    review_vector = np.array([get_sentence_vector_glove(processed_review, vectorizer)])
    review_vector = review_vector.reshape((1, 1, 100))  # ضبط الشكل ليتناسب مع LSTM

    prediction = model.predict(review_vector)
    sentiment = "🟢 Positive" if prediction[0] > 0.5 else "🔴 Negative"

    print(f"Processed Review: {processed_review}")
    print(f"Predicted Sentiment: {sentiment}")

    return sentiment

# 📝 تجربة النموذج على عدة جمل
test_reviews = [
    "I absolutely loved this movie! The story was fantastic and the acting was great.",
    "This movie was terrible. I regret watching it.",
    "One of the best movies I have ever seen, I highly recommend it!",
    "I couldn't even finish this movie, it was so bad and boring.",
    "The visuals were stunning, but the story was weak."
]

for review in test_reviews:
    sentiment = predict_sentiment(review, model, glove_model)
    print(f"Final Sentiment Prediction: {sentiment}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Processed Review: i absolutely loved movie story fantastic acting great
Predicted Sentiment: 🟢 Positive
Final Sentiment Prediction: 🟢 Positive
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Processed Review: movie terrible i regret watching
Predicted Sentiment: 🔴 Negative
Final Sentiment Prediction: 🔴 Negative
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Processed Review: one best movies i have ever seen i highly recommend
Predicted Sentiment: 🟢 Positive
Final Sentiment Prediction: 🟢 Positive
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Processed Review: i couldnt even finish movie so bad boring
Predicted Sentiment: 🔴 Negative
Final Sentiment Prediction: 🔴 Negative
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Processed Review: visuals were stunning but story weak
Predicted Sent

In [11]:
test_reviews = [
    "The acting was great, but the storyline was weak and predictable.",
    "I found this movie really disappointing, the visuals were nice but the plot was bad.",
    "The movie was terrible, a complete waste of time.",
    "This film was amazing, but the pacing was dull.",
    "One of the best movies ever, highly recommend!"
]

for review in test_reviews:
    sentiment = predict_sentiment(review, model, glove_model)
    print(f"Final Sentiment Prediction: {sentiment}")
    print("-" * 50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Processed Review: acting great but storyline weak predictable
Predicted Sentiment: 🔴 Negative
Final Sentiment Prediction: 🔴 Negative
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Processed Review: i found movie really disappointing visuals were nice but plot bad
Predicted Sentiment: 🔴 Negative
Final Sentiment Prediction: 🔴 Negative
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processed Review: movie terrible a complete waste time
Predicted Sentiment: 🔴 Negative
Final Sentiment Prediction: 🔴 Negative
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Processed Review: film amazing but pacing dull
Predicted Sentiment: 🟢 Positive
Final Sentiment Prediction: 🟢 Positive
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processed Review: one best movies ever highly recommend
Predicted Sentimen

In [12]:
# 🟢 جمل إيجابية (Positive)
positive_reviews = [
    "This movie was absolutely fantastic! The acting and storyline were top-notch.",
    "I loved every second of this film, truly a masterpiece.",
    "One of the best movies I’ve ever seen, highly recommended!",
    "The cinematography was breathtaking, and the music was perfect.",
    "An outstanding performance by the lead actor made this film unforgettable.",
    "The visuals were mesmerizing, and the story was deeply emotional.",
    "A movie filled with action and suspense, kept me on the edge of my seat.",
    "A heartwarming and inspiring story, loved it!",
    "The humor was spot on, I laughed the entire time!",
    "The movie was so engaging that I didn’t want it to end."
]

# 🔴 جمل سلبية (Negative)
negative_reviews = [
    "The worst movie I’ve ever seen. A complete waste of time.",
    "The plot was dull, predictable, and lacked originality.",
    "The acting was terrible, it felt like they weren’t even trying.",
    "I fell asleep halfway through, it was so boring.",
    "The dialogue was cringe-worthy and unnatural.",
    "This film was a huge disappointment, I expected much better.",
    "The visuals were good, but the story was weak and pointless.",
    "I regret watching this, it was a total disaster.",
    "The movie dragged on forever, and nothing interesting happened.",
    "The action scenes were so fake and over-exaggerated."
]

# 🤯 جمل محايدة أو غامضة (Neutral/Hard)
neutral_or_hard_reviews = [
    "The movie was okay, but it didn’t really stand out.",
    "I didn’t hate it, but I wouldn’t watch it again.",
    "There were some good moments, but overall it felt lacking.",
    "The film had potential, but it failed to deliver in the second half.",
    "A mix of good and bad elements, hard to say if I liked it.",
    "The visuals were great, but the story was underwhelming.",
    "Not the best, but not the worst either.",
    "I expected more from this director, but it wasn’t too bad.",
    "Some parts were amazing, but the rest was just average.",
    "The beginning was promising, but the ending ruined it."
]

# 🛠️ اختبار النموذج على جميع الجمل
test_reviews = positive_reviews + negative_reviews + neutral_or_hard_reviews

print("🚀 Running Sentiment Analysis Test...\n")
for review in test_reviews:
    sentiment = predict_sentiment(review, model, glove_model)
    print(f"📝 Processed Review: {review}")
    print(f"🎯 Final Sentiment Prediction: {sentiment}")
    print("-" * 70)


🚀 Running Sentiment Analysis Test...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Processed Review: movie absolutely fantastic acting storyline were topnotch
Predicted Sentiment: 🟢 Positive
📝 Processed Review: This movie was absolutely fantastic! The acting and storyline were top-notch.
🎯 Final Sentiment Prediction: 🟢 Positive
----------------------------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Processed Review: i loved every second film truly a masterpiece
Predicted Sentiment: 🟢 Positive
📝 Processed Review: I loved every second of this film, truly a masterpiece.
🎯 Final Sentiment Prediction: 🟢 Positive
----------------------------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Processed Review: one best movies ive ever seen highly recommended
Predicted Sentiment: 🟢 Positive
📝 Processed Review: One of the best movies I’ve ever seen, highly recommended!
🎯 Final Sentiment Prediction: 🟢 Positive
---------------------------